**The idea** is to match a song to some textual context based on valency, category and text similarity using song lyrics<br />
**Valency:** pos, neg, neu<br />
**Categories:** adventure, hobbies, humor, mystery, romance<br />
**Similarity metrics:** cosine, tf-idf, WordNet word similarity or others

In [1]:
import nltk, re, pprint
import pandas as pd
import numpy as np

#### EDA and Data Preprocessing:

Import the dataset:

In [2]:
data = pd.read_csv("D:\ML\Datasets\labeled_lyrics_cleaned.csv")

In [3]:
data.head()

Unnamed: 0        artist  \
0           0  Elijah Blake   
1           1  Elijah Blake   
2           2  Elijah Blake   
3           3  Elijah Blake   
4           4  Elijah Blake   

                                                 seq                song  \
0  No, no\r\nI ain't ever trapped out the bando\r...            Everyday   
1  The drinks go down and smoke goes up, I feel m...    Live Till We Die   
2  She don't live on planet Earth no more\r\nShe ...       The Otherside   
3  Trippin' off that Grigio, mobbin', lights low\...               Pinot   
4  I see a midnight panther, so gallant and so br...  Shadows & Diamonds   

   label  
0  0.626  
1  0.630  
2  0.240  
3  0.536  
4  0.371

Rename columns:

In [4]:
#data = data.round(decimals=2)
data.drop(labels="Unnamed: 0", axis=1, inplace=True)
data.rename(columns={"seq": "lyrics", "label": "valency"}, inplace=True)

In [5]:
data.head()

artist                                             lyrics  \
0  Elijah Blake  No, no\r\nI ain't ever trapped out the bando\r...   
1  Elijah Blake  The drinks go down and smoke goes up, I feel m...   
2  Elijah Blake  She don't live on planet Earth no more\r\nShe ...   
3  Elijah Blake  Trippin' off that Grigio, mobbin', lights low\...   
4  Elijah Blake  I see a midnight panther, so gallant and so br...   

                 song  valency  
0            Everyday    0.626  
1    Live Till We Die    0.630  
2       The Otherside    0.240  
3               Pinot    0.536  
4  Shadows & Diamonds    0.371

Check summary statistics:

In [6]:
data.describe(include='all')

artist                                             lyrics  \
count          158353                                             158353   
unique          14691                                             135991   
top     Elvis Presley  Somewhere over the rainbow, way up high\r\nThe...   
freq              821                                                167   
mean              NaN                                                NaN   
std               NaN                                                NaN   
min               NaN                                                NaN   
25%               NaN                                                NaN   
50%               NaN                                                NaN   
75%               NaN                                                NaN   
max               NaN                                                NaN   

                                          song        valency  
count                                   158353  158353.000000  
unique                                   99031            NaN  
top     Have Yourself a Merry Little Christmas            NaN  
freq                                       162            NaN  
mean                                       NaN       0.491052  
std                                        NaN       0.249619  
min                                        NaN       0.000000  
25%                                        NaN       0.286000  
50%                                        NaN       0.483000  
75%                                        NaN       0.691000  
max                                        NaN       0.998000

Remove cover songs and format decimal places for summary statistics display:

In [7]:
#data = data.drop_duplicates(subset=['lyrics', 'song'])
data.sort_values(by=['song', 'valency'], ascending=False, inplace=True) # to keep highest value valency 
data = data.drop_duplicates(subset='lyrics')
pd.set_option('display.float_format', lambda x: '%.2f' % x) # round everything to 2 decimal places

In [8]:
data.describe(include='all')

artist                                             lyrics  \
count          135991                                             135991   
unique          10777                                             135991   
top     Elvis Presley  In my deepest mood\r\nHear my call for you O' ...   
freq              753                                                  1   
mean              NaN                                                NaN   
std               NaN                                                NaN   
min               NaN                                                NaN   
25%               NaN                                                NaN   
50%               NaN                                                NaN   
75%               NaN                                                NaN   
max               NaN                                                NaN   

          song   valency  
count   135991 135991.00  
unique   95714       nan  
top      Intro       nan  
freq       127       nan  
mean       NaN      0.50  
std        NaN      0.25  
min        NaN      0.00  
25%        NaN      0.30  
50%        NaN      0.50  
75%        NaN      0.70  
max        NaN      1.00

Spot-checking three random entries to confirm data integrity:

In [9]:
data = data.reset_index() 
data

index               artist  \
0        54911        Simon &  Milo   
1        82479         Hippo Campus   
2        82478         Hippo Campus   
3        82477         Hippo Campus   
4        82476         Hippo Campus   
...        ...                  ...   
135986  109667       The Beach Boys   
135987   55096         Simple Minds   
135988   41838   The Blues Brothers   
135989   81217  The Gaslight Anthem   
135990   38621                   U2   

                                                   lyrics  \
0       Hello, this is Stacy, the computer\nGood morni...   
1       See how the western kids\r\nHave silicon insid...   
2       Wisconsin pines, collaborating with the day gl...   
3       I see meaning where you don't, where you don't...   
4       My thoughts are a battlefield of sub-surreal a...   
...                                                   ...   
135986  Hi, this is Al this scene takes place at a typ...   
135987  Cry cry cry\r\nCry like a baby\r\n"see" Moon "...   
135988  Caught a ride into South Dakota\r\nWith two gi...   
135989  Have you seen my hands?\nJust look at 'em shak...   
135990  I waited patiently for the Lord\nHe inclined a...   

                                     song  valency  
0                     www.nevergetoveryou     0.68  
1                            western kids     0.52  
2                             way it goes     0.52  
3                                   vines     0.66  
4                                vacation     0.55  
...                                   ...      ...  
135986  "Cassius" Love Vs. "Sonny" Wilson     0.49  
135987           "C" Moon Cry Like a Baby     0.77  
135988            "B" Movie Box Car Blues     0.50  
135989                               "45"     0.42  
135990                               "40"     0.32  

[135991 rows x 5 columns]

### Attempting sentiment classification using Vader:

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

#### Analysing valency and extracting compund score:

In [11]:
def valency(text):
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(text)
    #ss.pop('compound')
    compound_score = ss.get('compound')
    if compound_score > 0.3 and compound_score <= 1:
        valency = 'positive'
    elif compound_score >= -1 and compound_score < -0.3:
        valency = 'negative'
    else:
        valency = 'neutral'
    return valency, compound_score

Loading song lyrics:

In [12]:
lyrics = data.at[201, 'lyrics'].strip()
#text = tokenize.sent_tokenize(lyrics)[0]
print(lyrics[:100])

Every time I turn my back I get the feeling that
I'm 'bout to take a shot to the skully with a bat


In [13]:
valency(lyrics)

('negative', -0.8814)

User situtation test:

In [14]:
s1 = "Today is finally my day off! The weather is amazing and I'm going to the beach"
s2 = "Today is finally my day off! The weather is [] and I'm going to the beach"
valency(s1), valency(s2)

(('positive', 0.6239), ('neutral', 0.0))

As can be seen in the example above accuracy is not great so a diiferent classisifier is needed, possibly trained on NLTK moview reviews corpus.

If max score required instead:

In [15]:
#max_value = max(ss.values())
#max_value
#max_key = [k for k, v in ss.items() if v == max_value][0]
#max_key

### Training classifier to assing one of the Brown corpus categories to an arbitrary text:

In [16]:
from nltk.corpus import brown, stopwords
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

categories = ['adventure', 'hobbies', 'humor', 'mystery', 'romance']
#cfd = nltk.ConditionalFreqDist((genre, word) for genre in brown.categories() for word in brown.words(categories=genre))

In [17]:
fdist1 = nltk.FreqDist([lemmatizer.lemmatize(word) for word in brown.words(categories='humor')
                        if word.isalnum() and word.lower() not in stopwords.words('english')])

In [18]:
fdist1.most_common(10)

[('said', 87),
 ('one', 65),
 ('would', 56),
 ('time', 50),
 ('thing', 40),
 ('even', 38),
 ('like', 34),
 ('could', 30),
 ('way', 29),
 ('year', 29)]

In [19]:
fdist2 = nltk.FreqDist([lemmatizer.lemmatize(word) for word in brown.words(categories='mystery')
                    if word.isalnum() and word.lower() not in stopwords.words('english')])

In [20]:
fdist2.most_common(10)

[('said', 202),
 ('would', 186),
 ('one', 175),
 ('back', 157),
 ('could', 141),
 ('like', 136),
 ('man', 106),
 ('get', 101),
 ('know', 93),
 ('time', 87)]

In [21]:
dataset = []

for category in brown.categories():
    for fileid in brown.fileids(category):
        if category in categories:
            dataset.append((brown.words(fileids = fileid), category))

In [22]:
#dataset

### Extracting features from lyrics:

In [23]:
#tokens = [tokenize.sent_tokenize(x) for x in data['lyrics']]

In [24]:
#text = tokens[0][8]
#text